# Daten

## Datensätze und -quellen
In der letzten Übung haben Sie die Grundlagen von Python kennen gelernt und erste Analysen an einem Datensatz vorgenommen. Heute wollen wir darauf aufbauen und uns mehr mit medizinischen Daten auseinandersetzen. Der Datensatz von letzter Woche war von [Kaggle](https://www.kaggle.com/datasets). Kaggle bietet eine der größten öffentlichen Sammlungen von Datensätzen zu allen möglichen Themen und richtet sich explizit an Data Scientists und KI-Forscher. Das Unternehmen veranstaltet regelmäßig Wettbewerbe, bei denen der beste ML-Ansatz gesucht wird. Seit 2017 gehört Kaggle zu Google. 

Aus der Vorlesung kennen Sie außerdem bereits [PhysioNet](https://physionet.org/). Recherchieren Sie andere mögliche Datenquellen für Medizinische Daten. Lesen Sie den Datensatz *mystery_dataset_ex2.csv* zur genaueren Betrachtung ein und finden Sie den Datensatz online. Machen Sie sich damit vertraut, was die einzelnen Spalten aussagen.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
imported_dataset = pd.read_csv('/content/gdrive/MyDrive/mystery_dataset_ex2.csv')
imported_dataset

Andere Quellen von Datensätzen sind zum Beispiel:
* [Data.gov](https://data.gov) ist die Open Data Sammlung der US-Regierung. Hier kann man nach Themen filtern, unter anderem nach [Older Adults Health](https://catalog.data.gov/group/older-adults-health-data)
* Die [WHO](https://apps.who.int/gho/data/node.resources) stellt viele Statistiken zu den Gesundheitssystemen und der medizinischen Versorgung ihrer Mitgliedsstaaten zur Verfügung, zum Beispiel zu [COVID-19](https://covid19.who.int/table)
* [OASIS](https://www.oasis-brains.org/) ist eine Quelle speziell für Neuroimaging, also radiologisch erzeugte Bilder des Gehirns. Diese Bilder sind zwar frei verfügbar, allerdings muss man sich für die Nutzung registrieren.
* Registry of Research Data Repositories, oder kurz [RE3](https://www.re3data.org/) bietet eine sehr schön thematisch gegliederte Übersicht über verschiedenste Datensätze. Viele der Datensätze brauchen aber auch hier eine offizielle Anfrage zur Nutzung.

Der zur Verfügung gestellte Datensatz *mystery_dataset_ex2.csv* über Herz-Kreislauf-Erkrankungen stammt auch dieses mal wieder von Kaggle. Zu finden ist er [hier](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data?select=heart_failure_clinical_records_dataset.csv).

## Datenanalyse

Ein sehr wichtiger Schritt beim Machine Learning ist die genaue Analyse des Datensatzes. Das gilt insbesondere, wenn Sie ihn nicht selber erhoben haben.Schauen Sie sich zum Beispiel den Zusammenhang zwischen den Spalten *time* und *DEATH_EVENT* an. 

Tipp: 
Mit dem Befehl `pd.set_option('display.max_columns',None)` können Sie einstellen, dass die Tabelle die Spaltenanzahl nicht begrenzt. Nutzen Sie das, um sich die relevanten Spalten in Gänze anzuzeigen. Sie müssen die Tabelle dafür transponieren. Wenn Sie danach die ursprüngliche Tabellenformatierung wieder herstellen wollen, empfiehlt es sich, die aktuelle Spaltenanzahl vorher mit `pd.get_option('display.max_columns')` als Variable zu speichern.

In [ ]:
# Beziehung zwischen time und DEATH_EVENT dargestellt als Graph
import matplotlib.pyplot as plt
plt.scatter(imported_dataset['time'],imported_dataset['DEATH_EVENT'])

In [ ]:
# Beziehung zwischen time und DEATH_EVENT als Array:
displayed_nr_columns = pd.get_option('display.max_columns')
pd.set_option('display.max_columns',None)
imported_dataset[['time','DEATH_EVENT']].transpose()

In [ ]:
pd.set_option('display.max_columns',displayed_nr_columns)

In [ ]:
# lieber als Barplot:
import matplotlib.pyplot as plt
import numpy as np
bins = 10
time_length = len(imported_dataset['time'])
time_range = list(range(0,time_length,int(time_length/bins)))
probabilities = []
for i in range(len(time_range)-1):
  span = range(time_range[i],time_range[i+1])
  fraction = imported_dataset[imported_dataset['time'].isin(span)]
  probabilities.append(np.mean(fraction['DEATH_EVENT']))
t = list(range(10))
labels = ["< " + str(time_range[i]) for i in range(1,11)]
f = plt.figure(figsize=[12.8,9.6])
ax = f.add_subplot(111)
ax.bar(t,probabilities,tick_label=labels)

Was stellen Sie fest? Überprüfen Sie Ihren Verdacht, indem Sie die Korrelation dieser beiden Spalten berechnen.

In [ ]:
cor_time_death = imported_dataset[['time','DEATH_EVENT']].corr()
print("Korrelation zwischen den Spalten 'time' und 'DEATH_EVENT' beträgt:\n", cor_time_death)

Wenn Sie die Analyse des Datensatzes etwas strukturierter angehen wollen, stellt Pandas mehrere Befehle zur Verfügung, die Informationen über ein Dataframe liefern: `dataframe.shape`, `dataframe.dtypes`, `dataframe.describe()` und `dataframe.isnull()`. Führen Sie die Befehle aus und machen Sie sich damit vertraut, was sie ausgeben.

In [ ]:
imported_dataset.shape

In [ ]:
imported_dataset.dtypes

Wie Sie sehen, muss dtypes nicht immer aussagekräftig sein! Dass `'Anaemia'`, `'Diabetes'`, `'Sex'`, `'Smoking'` und `'DEATH_EVENT'` nominal skalierte kategoriale Daten und alle anderen numerische Daten sind, wird hier nicht ersichtlich!

In [ ]:
imported_dataset.describe()

Wenn Sie überprüfen wollen, ob Sie ungültige Werte (`NULL`) in Ihren Daten haben, gibt es außerdem noch diesen Befehl

In [ ]:
imported_dataset.isnull().sum()

Zurück zur Spalte *time*: Das beobachtete Phänomen bezeichnet man als **Data Leakage** und wird in einer späteren Vorlesung nochmal ausführlich behandelt. Für jetzt reicht es, wenn Sie den Datensatz bereinigen, indem Sie die Spalte *time* einfach löschen.

In [ ]:
# Löschen Sie die Spalte time
imported_dataset.drop(columns='time',inplace=True)

## Einfache Wahrscheinlichkeiten

Bestimmen Sie, wie viel Prozent der Patient:innen männlich und wie viel weiblich sind, sowie die Anzahl der Patient:innen, die während der Beobachtungsphase gestorben sind.

In [ ]:
print('Anteil männlicher Patienten: ', sum(imported_dataset['sex'])/len(imported_dataset))
print('Anteil weiblicher Patientinnen: ', 1-sum(imported_dataset['sex'])/len(imported_dataset))
print('Anzahl Patient:innen, die gestorben sind: ', sum(imported_dataset['DEATH_EVENT']))

Berechnen Sie jetzt, wie wahrscheinlich es ist, dass ein:e Patient:in gestorben ist. Berechnen Sie außerdem, wie wahrscheinlich es ist, dass ein Patient gestorben ist, wenn bekannt ist, dass er männlich war und wie wahrscheinlich es ist, dass eine Patientin gestorben ist, wenn bekannt ist, dass sie weiblich war.

In [ ]:
print('Wahrscheinlichkeit von Tod:  ', sum(imported_dataset['DEATH_EVENT'])/len(imported_dataset))
only_male_patients = imported_dataset[imported_dataset['sex']==1]
only_female_patients = imported_dataset[imported_dataset.sex.eq(0)]
print('Wahrscheinlichkeit von Tod, wenn bekannt dass männlich: ', sum(only_male_patients['DEATH_EVENT'])/len(only_male_patients))
print('Wahrscheinlichkeit von Tod, wenn bekannt dass weiblich: ', sum(only_female_patients['DEATH_EVENT'])/len(only_female_patients))

Berechnen Sie, wie wahrscheinlich es ist, dass ein:e Patient:in gestorben ist, wenn bekannt war, dass er/sie raucht. Berechnen Sie auch diese Wahrscheinlichkeit in Abhängigkeit der Geschlechter.

In [ ]:
smokers = imported_dataset[imported_dataset['smoking']==1]
male_smokers = only_male_patients[only_male_patients['smoking']==1]
female_smokers = only_female_patients[only_female_patients['smoking']==1]
print('Wahrscheinlichkeit von Tod, wenn bekannt, dass Raucher:in: ', sum(smokers['DEATH_EVENT'])/len(smokers))
print('Wahrscheinlichkeit von Tod, wenn bekannt, dass männlich und Raucher: ', sum(male_smokers['DEATH_EVENT'])/len(male_smokers))
print('Wahrscheinlichkeit von Tod, wenn bekannt, dass weiblich und Raucher: ', sum(female_smokers['DEATH_EVENT'])/len(female_smokers))

Überrascht Sie das Ergebnis? Können Sie erklären, wie es zustande kommt?

In [ ]:
len(female_smokers)

Angenommen, Sie wollen wissen, welche der Features gute Prädiktoren für den Tod des/der Patient:in sind. Welche Größen können Sie berechnen, um dafür ein Gefühl zu bekommen? Welche Features würden Sie für eine Prädiktion empfehlen? 

In [ ]:
# probieren Sie sich ein bisschen aus!
imported_dataset.corr()

**Paper zum Datensatz:**

[Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone.](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5)

-> stimmt mit unseren Überlegungen überein!